In [1]:
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad, elementwise_grad, hessian, jacobian

import sys
sys.path.insert(0, "../../project_2/src")
from SGD import minibatch

In [125]:
# Inspired by mortens example code in week 43

def activation_function(z):
    return np.maximum(0, z)

def activation_out(z):
    return z

"""
network_shape = [inputs, w1, w2, ..., wL, outputs]

W = wwwwb
    wwwwb
"""
def initialize_params(network_shape):
    P = []
    for i in range(1, len(network_shape)):
        k = network_shape[i-1]
        j = network_shape[i]
        P.append(np.random.randn(j, k) * np.sqrt(2) / np.sqrt(k))
        P[i-1] = np.concatenate((P[i-1], np.zeros(j).reshape(-1,1)), axis=1)

    return P

def Network(t, P):
    # Assumes the input t to be a scalar, returns a 2d-row vector e.g. shape=[1,6]
    a = t.reshape(-1,1)
    for P_i in P:
        #a = np.concatenate((a, np.ones(np.size(a, 0)).reshape(-1, 1)), axis=1) 
        a = np.concatenate((a, np.ones((1,1))), axis=1) 
        z = np.matmul(a, np.transpose(P_i))
        a = activation_function(z)
    
    return activation_out(z) 

def Network_predict(t, P):
    # For predictions
    # Assumes the input t to be a 1d-array, 
    # returns a matrix where each row corresponds to a vector for a particular t
    a = t.reshape(-1,1)
    for P_i in P:
        a = np.concatenate((a, np.ones(np.size(a, 0)).reshape(-1, 1)), axis=1) 
        #a = np.concatenate((a, np.ones((1,1))), axis=1) 
        z = np.matmul(a, np.transpose(P_i))
        a = activation_function(z)
    
    return activation_out(z) 

def optimize(t, P, A, x_0, N_minibatches, learning_rate, n_epochs):
    # Assumes t is a 1d-array.
    assert N_minibatches <= np.size(t, 0)

    cost_func_grad = grad(costfunction, 1) # Check which grad-call is correct.
    
    for epoch in range(n_epochs):
        mb = minibatch(t, N_minibatches)
        for i in range(N_minibatches):
            t_mb = t[mb[i]]
            M = np.size(t_mb, 0)
            # compute gradients of weights
            cost_grad = cost_func_grad(t_mb, P, A, x_0)
            for l in range(len(P)):
                P[l] -= learning_rate * cost_grad[l]
    return P

def g_trial(t, P, x_0):
    #assumes the input t to be a scalar, x_0 is 1d-row-vector which broadcasts to e.g. (1,6)
    
    return np.exp(-t)*x_0 + (1-np.exp(-t))*Network(t,P)

def g_trial_predict(t, P, x_0):
    # For predictions
    #assumes the input t to be a 1d-array, broadcasts along the rows of Network_predict's output
    t = t.reshape(-1,1)
    return np.exp(-t)*x_0 + (1-np.exp(-t))*Network_predict(t,P)

def costfunction(t, P, A, x_0):
    
    cost = 0
    g_grad = jacobian(g_trial,0) # Check that this is the correct grad-call
    
    for time in t:
        d_dt = g_grad(time,P,x_0).reshape(-1,1) # should have shape (eigenvector_length,1)
        x_t = g_trial(time,P,x_0).reshape(-1,1) #check shape, should be (eigenvector_length,1)
        # Right hand side is: -x + [x.T @ x * A + (1 - x.T @ A @ x) @ I] @ x where x is x_t is a column vector
        right_side = (-x_t + (x_t.T @ x_t * A + (1 - x_t.T @ A @ x_t) * np.identity(np.size(x_t,0))) @ x_t)
        print(x_t)
        print(right_side)
#         right_side = -x_t + np.matmul( # -x + [
#                                 np.matmul(np.transpose(x_t), x_t) * A #(x.T @ x) * A
#                                 + ( 1 - np.matmul(np.transpose(x_t),np.matmul(A, x_t))) # + (1-x.t @ A @ x)
#                                 * np.identity(np.size(x_t, 0)) # * I
#                                 , x_t) # ] @ x
        cost = cost + np.sum((d_dt - right_side)**2) / np.size(x_t) 
                                                              
        

    return cost / np.size(t)

In [126]:
t = np.linspace(0,1,100)
Q = np.random.randn(6,6)
A = (np.transpose(Q) + Q) /2 #standard trick for making symmetric real
A = np.array([[0.7663, 0.4283, -0.3237, -0.4298, -0.1438], [0.4283,0.2862,0.0118,-0.2802,0.1230]
              , [-0.3237,0.0118,-0.9093,-0.4383,0.7684],[-0.4298,-0.2802,-0.4384,-0.0386,-0.1315]
              , [-0.1438,0.1230,0.7684,-0.1315,-0.4480]]) #matrix from Yi et al.

x_0 = np.random.randn(np.size(A,0)) #initial guess for eigenvector.

network_shape = [1, 10, np.size(x_0)] #output must match eigenvector length
P = initialize_params(network_shape)

In [127]:
P = optimize(t, P, A, x_0,10, 0.002, 10)
#y = Network(X[0, :].reshape(1, 2), P)

Autograd ArrayBox with value [[-1.9655194 ]
 [-0.67214837]
 [ 0.62936239]
 [-1.3038906 ]
 [ 2.52230815]]
Autograd ArrayBox with value [[-14.98374128]
 [ -1.74681949]
 [ 30.20747831]
 [ 11.3913726 ]
 [-13.79026715]]
Autograd ArrayBox with value [[-1.36491279]
 [-0.58877403]
 [ 0.61667982]
 [-1.1576762 ]
 [ 1.65741722]]
Autograd ArrayBox with value [[-4.864249  ]
 [ 0.01614237]
 [ 9.51279917]
 [ 4.94489343]
 [-4.08559606]]
Autograd ArrayBox with value [[-0.36490371]
 [-0.58776842]
 [ 0.79845337]
 [-1.19910191]
 [ 0.1110404 ]]
Autograd ArrayBox with value [[-0.68816572]
 [ 0.18074382]
 [-0.13520861]
 [ 0.1959259 ]
 [ 1.78326236]]
Autograd ArrayBox with value [[-0.35655928]
 [-0.59613129]
 [ 0.81229447]
 [-1.21675183]
 [ 0.0916777 ]]
Autograd ArrayBox with value [[-0.69732633]
 [ 0.17467147]
 [-0.16978388]
 [ 0.14751978]
 [ 1.88593753]]
Autograd ArrayBox with value [[-0.66173582]
 [-0.53577368]
 [ 0.66751063]
 [-1.07871015]
 [ 0.61039947]]
Autograd ArrayBox with value [[-0.95847822]
 [ 0.3

Autograd ArrayBox with value [[-0.29286824]
 [-0.5411038 ]
 [ 0.81020897]
 [-1.1681676 ]
 [ 0.0484035 ]]
Autograd ArrayBox with value [[-0.52700856]
 [ 0.16759852]
 [-0.26408786]
 [-0.05607707]
 [ 1.75200285]]
Autograd ArrayBox with value [[-0.58971744]
 [-0.17221914]
 [ 0.94828627]
 [-0.82599831]
 [ 0.73312937]]
Autograd ArrayBox with value [[-0.78051958]
 [ 0.27436802]
 [-0.45640048]
 [ 0.49971764]
 [ 0.58997725]]
Autograd ArrayBox with value [[-0.3436329 ]
 [-0.33083355]
 [ 0.78712814]
 [-0.93202255]
 [ 0.275138  ]]
Autograd ArrayBox with value [[-0.41244869]
 [ 0.23008632]
 [-0.26074184]
 [ 0.14526256]
 [ 0.99955136]]
Autograd ArrayBox with value [[-0.74302018]
 [-0.11432899]
 [ 1.06258635]
 [-0.81143363]
 [ 0.98727067]]
Autograd ArrayBox with value [[-1.24271236]
 [ 0.23013448]
 [-0.52944784]
 [ 0.67962897]
 [ 0.21980652]]
Autograd ArrayBox with value [[-0.47706974]
 [-0.22821524]
 [ 0.86887055]
 [-0.8536327 ]
 [ 0.53613302]]
Autograd ArrayBox with value [[-0.56357371]
 [ 0.270351

Autograd ArrayBox with value [[-0.29745762]
 [-0.39298181]
 [ 0.76441304]
 [-0.9977966 ]
 [ 0.16264577]]
Autograd ArrayBox with value [[-0.38214233]
 [ 0.20429132]
 [-0.22616781]
 [ 0.04795969]
 [ 1.15189753]]
Autograd ArrayBox with value [[-0.41402884]
 [-0.26469117]
 [ 0.81778337]
 [-0.88657731]
 [ 0.41668645]]
Autograd ArrayBox with value [[-0.46166941]
 [ 0.25737096]
 [-0.2906539 ]
 [ 0.27240342]
 [ 0.85478657]]
Autograd ArrayBox with value [[-0.38617483]
 [-0.28605921]
 [ 0.80171074]
 [-0.90169362]
 [ 0.36295928]]
Autograd ArrayBox with value [[-0.43363096]
 [ 0.24847428]
 [-0.27181069]
 [ 0.22716019]
 [ 0.89917367]]
Autograd ArrayBox with value [[-0.28521051]
 [-0.42629921]
 [ 0.76575713]
 [-1.0336361 ]
 [ 0.1211646 ]]
Autograd ArrayBox with value [[-0.39016182]
 [ 0.19273813]
 [-0.22678198]
 [ 0.00740802]
 [ 1.25616699]]
Autograd ArrayBox with value [[-0.28211166]
 [-0.47169268]
 [ 0.77686104]
 [-1.08267934]
 [ 0.08478523]]
Autograd ArrayBox with value [[-0.42729412]
 [ 0.180397

 [ 0.22863708]]
Autograd ArrayBox with value [[-0.37287724]
 [ 0.22166261]
 [-0.2159735 ]
 [ 0.1170456 ]
 [ 1.0191052 ]]
Autograd ArrayBox with value [[-0.77734902]
 [-0.10461691]
 [ 1.04072088]
 [-0.86119333]
 [ 1.01313695]]
Autograd ArrayBox with value [[-1.2155372 ]
 [ 0.24077516]
 [-0.27220179]
 [ 0.87640231]
 [ 0.11679606]]
Autograd ArrayBox with value [[-0.28053453]
 [-0.4511125 ]
 [ 0.76538797]
 [-1.06134924]
 [ 0.0967474 ]]
Autograd ArrayBox with value [[-0.39922536]
 [ 0.18496607]
 [-0.22120641]
 [-0.01129049]
 [ 1.33098752]]
Autograd ArrayBox with value [[-0.57308435]
 [-0.17696245]
 [ 0.90136935]
 [-0.8607236 ]
 [ 0.68753215]]
Autograd ArrayBox with value [[-0.66975066]
 [ 0.28389333]
 [-0.31393836]
 [ 0.5401759 ]
 [ 0.60263567]]
Autograd ArrayBox with value [[-0.4488756 ]
 [-0.23978254]
 [ 0.82388121]
 [-0.88318991]
 [ 0.47562062]]
Autograd ArrayBox with value [[-0.47942556]
 [ 0.26808441]
 [-0.27139422]
 [ 0.34314428]
 [ 0.78999453]]
Autograd ArrayBox with value [[-0.32719

Autograd ArrayBox with value [[-0.61316954]
 [-0.15816735]
 [ 0.91507368]
 [-0.87464549]
 [ 0.74603079]]
Autograd ArrayBox with value [[-0.72062548]
 [ 0.2890716 ]
 [-0.26163379]
 [ 0.6322904 ]
 [ 0.53121163]]
Autograd ArrayBox with value [[-0.45102504]
 [-0.23397703]
 [ 0.81555646]
 [-0.89053344]
 [ 0.47737478]]
Autograd ArrayBox with value [[-0.46388268]
 [ 0.2707231 ]
 [-0.24369303]
 [ 0.35719289]
 [ 0.777079  ]]
Autograd ArrayBox with value [[-0.3242228 ]
 [-0.33660754]
 [ 0.75503573]
 [-0.95421462]
 [ 0.23561856]]
Autograd ArrayBox with value [[-0.35989184]
 [ 0.22432383]
 [-0.20340086]
 [ 0.1269268 ]
 [ 0.99106912]]
Autograd ArrayBox with value [[-0.27815602]
 [-0.47485018]
 [ 0.76986085]
 [-1.08854409]
 [ 0.07686439]]
Autograd ArrayBox with value [[-0.41517578]
 [ 0.17833298]
 [-0.22309226]
 [-0.02943322]
 [ 1.41689078]]
Autograd ArrayBox with value [[-0.28027475]
 [-0.42341002]
 [ 0.75366103]
 [-1.03486979]
 [ 0.11509625]]
Autograd ArrayBox with value [[-0.3683537 ]
 [ 0.191943

...

 [ 0.28810708]]
Autograd ArrayBox with value [[-0.32100417]
 [ 0.24170156]
 [-0.13696817]
 [ 0.19383252]
 [ 0.85676933]]
Autograd ArrayBox with value [[-0.27908305]
 [-0.37542293]
 [ 0.71899457]
 [-0.99189557]
 [ 0.15554903]]
Autograd ArrayBox with value [[-0.30353598]
 [ 0.20593615]
 [-0.15572123]
 [ 0.05614715]
 [ 1.0302619 ]]
Autograd ArrayBox with value [[-0.35398197]
 [-0.29060833]
 [ 0.73354132]
 [-0.934436  ]
 [ 0.31878645]]
Autograd ArrayBox with value [[-0.33106012]
 [ 0.24955375]
 [-0.13421375]
 [ 0.22578485]
 [ 0.83054317]]
Autograd ArrayBox with value [[-0.49705482]
 [-0.21507595]
 [ 0.80214744]
 [-0.92202172]
 [ 0.56501096]]
Autograd ArrayBox with value [[-0.4744776 ]
 [ 0.30769672]
 [-0.0909053 ]
 [ 0.50614469]
 [ 0.65473533]]
Autograd ArrayBox with value [[-0.27408499]
 [-0.38617378]
 [ 0.72042683]
 [-1.00146056]
 [ 0.14079463]]
Autograd ArrayBox with value [[-0.30574038]
 [ 0.20167398]
 [-0.1595787 ]
 [ 0.04035375]
 [ 1.0615449 ]]
Autograd ArrayBox with value [[-0.27391

Autograd ArrayBox with value [[-0.29504967]
 [-0.34620434]
 [ 0.71599337]
 [-0.96792443]
 [ 0.20016998]]
Autograd ArrayBox with value [[-0.29921507]
 [ 0.21841851]
 [-0.14388791]
 [ 0.10268514]
 [ 0.94793622]]
Autograd ArrayBox with value [[-0.26661157]
 [-0.41840675]
 [ 0.72734526]
 [-1.02912399]
 [ 0.10930932]]
Autograd ArrayBox with value [[-0.32334634]
 [ 0.19101242]
 [-0.16967956]
 [ 0.00876101]
 [ 1.15401684]]
Autograd ArrayBox with value [[-0.28202252]
 [-0.36613552]
 [ 0.71599605]
 [-0.9836667 ]
 [ 0.16770463]]
Autograd ArrayBox with value [[-0.29843943]
 [ 0.20939295]
 [-0.15006966]
 [ 0.06879971]
 [ 0.99952412]]
Autograd ArrayBox with value [[-0.26613407]
 [-0.42485371]
 [ 0.72962188]
 [-1.035085  ]
 [ 0.10412345]]
Autograd ArrayBox with value [[-0.32878848]
 [ 0.18913051]
 [-0.17241491]
 [ 0.00360054]
 [ 1.17529272]]
Autograd ArrayBox with value [[-0.53755904]
 [-0.20105871]
 [ 0.82325405]
 [-0.92847489]
 [ 0.63207423]]
Autograd ArrayBox with value [[-0.53188278]
 [ 0.324072

 [ 0.03719422]]
Autograd ArrayBox with value [[-0.84699892]
 [ 0.18096798]
 [-0.32737739]
 [-0.01512407]
 [ 2.5945151 ]]
Autograd ArrayBox with value [[-0.40248158]
 [-0.2596666 ]
 [ 0.74802409]
 [-0.92537317]
 [ 0.41340249]]
Autograd ArrayBox with value [[-0.36446501]
 [ 0.27515483]
 [-0.10528088]
 [ 0.32971794]
 [ 0.74654324]]
Autograd ArrayBox with value [[-0.4801606 ]
 [-0.22495228]
 [ 0.78683043]
 [-0.92591963]
 [ 0.54392569]]
Autograd ArrayBox with value [[-0.44986249]
 [ 0.30802579]
 [-0.07012468]
 [ 0.48482648]
 [ 0.65702254]]
Autograd ArrayBox with value [[-0.31080446]
 [-0.32618277]
 [ 0.71464362]
 [-0.95341457]
 [ 0.23957259]]
Autograd ArrayBox with value [[-0.30027479]
 [ 0.229263  ]
 [-0.13090109]
 [ 0.14476346]
 [ 0.88917565]]
Autograd ArrayBox with value [[-0.37164694]
 [-0.27751029]
 [ 0.73459553]
 [-0.92971965]
 [ 0.35844387]]
Autograd ArrayBox with value [[-0.3381157 ]
 [ 0.2608748 ]
 [-0.11449404]
 [ 0.26940569]
 [ 0.78482119]]
Autograd ArrayBox with value [[-0.37757

 [ 0.1762671 ]]
Autograd ArrayBox with value [[-0.28968424]
 [ 0.21154779]
 [-0.13771956]
 [ 0.07978676]
 [ 0.96629612]]
Autograd ArrayBox with value [[-0.58474797]
 [-0.19604392]
 [ 0.84213478]
 [-0.94596718]
 [ 0.71199281]]
Autograd ArrayBox with value [[-0.61378445]
 [ 0.35179348]
 [ 0.04231722]
 [ 0.72501688]
 [ 0.50599265]]
Autograd ArrayBox with value [[-0.3353509 ]
 [-0.63460988]
 [ 0.87566474]
 [-1.30080219]
 [ 0.0344244 ]]
Autograd ArrayBox with value [[-0.7629759 ]
 [ 0.16940185]
 [-0.3062083 ]
 [-0.0292073 ]
 [ 2.37570793]]
Autograd ArrayBox with value [[-0.61996696]
 [-0.18767115]
 [ 0.86300264]
 [-0.95443662]
 [ 0.76582822]]
Autograd ArrayBox with value [[-0.68313171]
 [ 0.36399568]
 [ 0.08901298]
 [ 0.81170135]
 [ 0.44747822]]
Autograd ArrayBox with value [[-0.27655254]
 [-0.5212146 ]
 [ 0.77793223]
 [-1.14026596]
 [ 0.0487124 ]]
Autograd ArrayBox with value [[-0.45110766]
 [ 0.16796109]
 [-0.2207146 ]
 [-0.05040298]
 [ 1.58105534]]
Autograd ArrayBox with value [[-0.34050

 [ 0.03353771]]
Autograd ArrayBox with value [[-0.65363057]
 [ 0.16506032]
 [-0.27988847]
 [-0.04745671]
 [ 2.11373894]]
Autograd ArrayBox with value [[-0.26339759]
 [-0.4618724 ]
 [ 0.73953013]
 [-1.06956799]
 [ 0.07789848]]
Autograd ArrayBox with value [[-0.357667  ]
 [ 0.17880217]
 [-0.18158645]
 [-0.02065743]
 [ 1.2910302 ]]
Autograd ArrayBox with value [[-0.3336901 ]
 [-0.30294226]
 [ 0.7150078 ]
 [-0.93907057]
 [ 0.29473349]]
Autograd ArrayBox with value [[-0.30600462]
 [ 0.244471  ]
 [-0.11094213]
 [ 0.20230709]
 [ 0.81855307]]
Autograd ArrayBox with value [[-0.2865369 ]
 [-0.54977008]
 [ 0.79892503]
 [-1.17787357]
 [ 0.04014117]]
Autograd ArrayBox with value [[-0.50861701]
 [ 0.16567048]
 [-0.23978038]
 [-0.05672605]
 [ 1.74617653]]
Autograd ArrayBox with value [[-0.42100447]
 [-0.25195009]
 [ 0.75124956]
 [-0.92615237]
 [ 0.45324056]]
Autograd ArrayBox with value [[-0.37938047]
 [ 0.28797354]
 [-0.07738463]
 [ 0.37625877]
 [ 0.70479616]]
Autograd ArrayBox with value [[-0.35500

In [118]:
g_trial_predict(t,P,x_0)[-1]

array([ 0.88161257, -0.02216758, -0.19547326,  0.31387196, -0.15481212])

In [78]:
# Solution from np.linalg.eigh

eigval, eigvec = np.linalg.eigh(A)
print(eigval)
print(eigvec)

[-1.56887992 -0.48134608 -0.03749509  0.51368892  1.23063218]
[[-0.11591938  0.36630714 -0.31514082 -0.31481908  0.80867607]
 [-0.03697112 -0.06476226  0.82605531  0.3081914   0.46592883]
 [-0.81391888  0.16545258 -0.18429373  0.52208713 -0.06018593]
 [-0.22777979  0.66454556  0.41248819 -0.46070518 -0.35227801]
 [ 0.52043083  0.6266016  -0.1191982   0.56664166 -0.0350885 ]]


In [95]:
#Forward Euler for Yi et al diff.eq

x_t = x_0.reshape(-1,1)
dt = 0.01
print(x_t) # initial guess
for i in range(1000):
    x_t = x_t + (-x_t + (x_t.T @ x_t * A + (1 - x_t.T @ A @ x_t) * np.identity(5)) @ x_t)*dt
print(x_t/np.sqrt(x_t.T @ x_t)) # normalized eigenvector
print(x_t.T @ A @ x_t / (x_t.T @ x_t)) # eigenvalue

[[ 0.83636627]
 [-0.53140545]
 [-2.09136711]
 [ 1.15610897]
 [-0.49678224]]
[[ 0.8086801 ]
 [ 0.46592499]
 [-0.06020916]
 [-0.35226867]
 [-0.03510045]]
[[1.2306343]]
